#Collaborative Filltering: Sistema de Recomendación de Películas basado en Factorización Matricial


Este proyecto implementa un sistema de recomendación de películas cuyo objetivo principal es predecir las preferencias cinematográficas de los usuarios y ofrecer recomendaciones personalizadas. El sistema funciona analizando el historial de calificaciones de los usuarios para identificar patrones de preferencia y sugerir películas que aún no han visto pero que probablemente disfrutarían.
Los objetivos específicos del sistema son:

Modelar matemáticamente las preferencias de los usuarios a partir de sus calificaciones previas
Descubrir factores latentes que influyen en las preferencias cinematográficas
Generar recomendaciones personalizadas para cada usuario
Abordar el problema de la escasez de datos (matriz dispersa) mediante técnicas de factorización

###Análisis del Dataset
El sistema utiliza un conjunto de datos de calificaciones de películas con las siguientes características:

Tamaño del dataset: 100,836 calificaciones
Número de usuarios: 610 usuarios únicos
Número de películas: 9,724 películas únicas
Escala de calificación: De 0.5 a 5.0 (normalizada a [0,1] durante el procesamiento)

El dataset presenta una matriz altamente dispersa, ya que cada usuario solo ha calificado una pequeña fracción del total de películas disponibles.

El dataset se encuentra en el siguiente link https://grouplens.org/datasets/movielens/. La versión utilizada es el que es más pequeño, llamado ml-latest-small.zip.

###Factorización Matricial con Sesgos
El algoritmo implementado se basa en una factorización matricial que representa tanto a usuarios como a películas en un espacio vectorial latente de 50 dimensiones. Esta técnica asume que existen factores subyacentes que determinan cómo los usuarios valoran las películas.

####Captura de Sesgos Individuales
Una característica importante del modelo es la captura explícita de sesgos

Sesgo global: Representa la calificación media del conjunto de datos.
Sesgos de usuario: Captura la tendencia de ciertos usuarios a calificar consistentemente más alto o más bajo que el promedio.
Sesgos de película: Refleja la calidad (en base a puntuaciones) inherente de cada película. Hay películas que tienden a ser puntuadas como mejores que otras.


In [1]:

import pandas as pd
import numpy as np



df = pd.read_csv('ratings.csv')
df = df.drop('timestamp', axis=1)
nombresDePelis = pd.read_csv('movies.csv')

df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [2]:
nombresDePelis.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df.shape[0]


100836

In [4]:
usuarios = df['userId'].unique()

cantidadDeUsuarios = len(usuarios)
maxUsuario = max(usuarios)


peliculas = df['movieId'].unique()

cantidadDePeliculas = len(peliculas)
maxPelicula = max(peliculas)

print(f"Cantidad de usuarios: {cantidadDeUsuarios}")
print(f"Maximo usuario: {maxUsuario}")

print(f"Cantidad de películas: {cantidadDePeliculas}")
print(f"Maxima película: {maxPelicula}")



Cantidad de usuarios: 610
Maximo usuario: 610
Cantidad de películas: 9724
Maxima película: 193609


In [5]:
mapeo_peliculasAIndice = {id_original: idx for idx, id_original in enumerate(peliculas)}
mapeo_IndiceAPelicula = {idx: id_original for id_original, idx in mapeo_peliculasAIndice.items()}


df['indiceNuevo'] = df['movieId'].map(mapeo_peliculasAIndice)


In [6]:
matrizDeRankings = np.full((cantidadDeUsuarios, cantidadDePeliculas), 0)
matrizBinaria = np.full((cantidadDeUsuarios, cantidadDePeliculas), 0)



In [7]:
for index, row in df.iterrows():
    usr = int(row['userId'])
    pel = int(row['movieId'])
    peliIndice = mapeo_peliculasAIndice[pel]
    rating = row['rating']
    matrizDeRankings[usr-1, peliIndice] = rating
    matrizBinaria[usr-1, peliIndice] = 1

In [8]:
Y_max = np.max(matrizDeRankings[matrizDeRankings > 0])
Y_min = np.min(matrizDeRankings[matrizDeRankings > 0])
matrizDeRankings = (matrizDeRankings - Y_min) / (Y_max - Y_min)

In [9]:
cantidadDeFeatures = 50


np.random.seed(42)
usuariosFeat = np.random.rand(cantidadDeUsuarios,cantidadDeFeatures) * 0.01
peliculasFeat = np.random.rand(cantidadDePeliculas,cantidadDeFeatures) * 0.01

print(usuariosFeat)
print(peliculasFeat)




[[3.74540119e-03 9.50714306e-03 7.31993942e-03 ... 5.20068021e-03
  5.46710279e-03 1.84854456e-03]
 [9.69584628e-03 7.75132823e-03 9.39498942e-03 ... 4.27541018e-03
  2.54191267e-04 1.07891427e-03]
 [3.14291857e-04 6.36410411e-03 3.14355981e-03 ... 5.02679023e-03
  5.14787512e-04 2.78646464e-03]
 ...
 [2.24494872e-03 6.58842900e-03 7.78588008e-03 ... 5.89793000e-03
  6.32507649e-03 9.21875920e-03]
 [9.29866484e-03 6.64249784e-03 7.75427817e-05 ... 4.05671453e-03
  7.56526020e-04 5.98670258e-03]
 [6.41701471e-03 8.16405928e-03 2.88576853e-03 ... 4.42299275e-03
  3.77180188e-03 3.33037264e-03]]
[[9.64944815e-03 7.71973411e-03 2.01959408e-04 ... 9.11655339e-03
  9.22184757e-04 9.42888796e-03]
 [4.58515206e-03 9.92625473e-03 5.39594654e-03 ... 8.26922153e-03
  3.12593601e-03 5.01787769e-04]
 [4.82596438e-03 9.13717144e-03 7.90468261e-03 ... 4.30642082e-03
  9.58578071e-04 7.46132090e-04]
 ...
 [1.92982647e-04 2.57546462e-03 9.18072257e-03 ... 1.90265277e-03
  6.11540216e-06 3.25719400e-03]

In [10]:
## Vamos a considerar los sesgos
# hay usuarios que tienenden a puntuar más alto
# hay películas objetivamente mejores que otras

#sesgo de película
mu = df['rating'].mean()
movie_means = df.groupby('indiceNuevo')['rating'].mean()
sesgoPelícula = movie_means - mu


print(sesgoPelícula)


#sesgo de usuarios
df['rating_adj'] = df.apply(lambda row: row['rating'] - mu - sesgoPelícula[row['indiceNuevo']], axis=1)
# Luego, calcula los promedios por usuario de estos ratings ajustados
user_means_adj = df.groupby('userId')['rating_adj'].mean()
sesgoUsuarios = user_means_adj





indiceNuevo
0       0.419373
1      -0.241942
2       0.444521
3       0.473812
4       0.736188
          ...   
9719   -1.001557
9720    0.998443
9721   -0.501557
9722   -0.001557
9723   -0.001557
Name: rating, Length: 9724, dtype: float64


###Optimización mediante Descenso de Gradiente
El sistema utiliza descenso de gradiente para encontrar los parámetros óptimos (vectores latentes y sesgos) que minimizan el error de predicción. Los hiperparámetros utilizados son:


Un aspecto fundamental de la implementación es la incorporación de regularización L2 en la función de costo. Esta técnica permite controlar la complejidad del modelo penalizando valores de parámetros excesivamente grandes. En nuestro caso, aplicamos un término de regularización (λ=0.01) a todos los parámetros del modelo: las matrices de factores latentes de usuario y película, así como los vectores de sesgo. Esta estrategia resultó crucial para evitar el sobreajuste, especialmente considerando la alta dispersión de nuestra matriz de calificaciones y la cantidad relativamente limitada de datos por usuario. La regularización garantiza que el modelo capture patrones generalizables en lugar de memorizar idiosincrasias específicas del conjunto de entrenamiento, mejorando así su capacidad predictiva en datos no observados.

In [11]:

  #  X: matriz de características de usuarios
  #  W: matriz de características de películas
  #  Y: matriz de rankings observados
  #  R: matriz binaria de existencia de rankings
  #  b_u: sesgo de usuario
  #  b_i: sesgo de película
  #  mu: promedio de película

def cost_function(X, W, b_u, b_i, mu, R, Y, lambda_reg):

    predictions = mu + b_u.reshape(-1, 1) + b_i.reshape(1, -1) + (X @ W.T)

    # Error cuadrático multiplicado por R para considerar solo ratings existentes
    error = (predictions - Y) * R

    cost_without_reg = 0.5 * np.sum(error ** 2)
    reg_term = 0.5 * lambda_reg * (np.sum(X ** 2) + np.sum(W ** 2) +
                                   np.sum(b_u ** 2) + np.sum(b_i ** 2))

    J = cost_without_reg + reg_term

    return J



In [12]:
def gradient_descent(X, W, b_u, b_i,
                     mu, R, Y, lambda_reg,
                     alpha, iterations):
    """
    Parámetros:
    - lambda_reg: parámetro de regularización
    - alpha: tasa de aprendizaje
    """
    historialDeCosto = []

    for i in range(iterations):
        # Calcular predicciones incluyendo sesgos
        predictions = mu + b_u.reshape(-1, 1) + b_i.reshape(1, -1) + (X @ W.T)
        # Calcular error solo en rankings existentes
        error = (predictions - Y) * R

        # Calcular gradientes
        X_grad = error @ W + lambda_reg * X
        W_grad = error.T @ X + lambda_reg * W
        b_u_grad = np.sum(error, axis=1) + lambda_reg * b_u
        b_i_grad = np.sum(error, axis=0) + lambda_reg * b_i

        # Actualizar parámetros
        X = X - alpha * X_grad
        W = W - alpha * W_grad
        b_u = b_u - alpha * b_u_grad
        b_i = b_i - alpha * b_i_grad

        # Calcular y almacenar costo
        J = cost_function(X, W, b_u, b_i, mu, R, Y, lambda_reg)
        historialDeCosto.append(J)

        if (i + 1) % 100 == 0:
            print(f"Iteración {i+1}/{iterations}, Costo: {J}")

    return X, W, b_u, b_i, historialDeCosto

In [13]:


X_optimizado, W_optimizado, b_u_optimizado, b_i_optimizado, historialDeCosto = gradient_descent(
    usuariosFeat, peliculasFeat, sesgoUsuarios.values, mu,
    sesgoPelícula.values, matrizBinaria, matrizDeRankings,
    lambda_reg=0.01, alpha=0.0005, iterations=5000
)



Iteración 100/5000, Costo: 7491.769893887463
Iteración 200/5000, Costo: 4933.583289131133
Iteración 300/5000, Costo: 4123.943882420781
Iteración 400/5000, Costo: 3649.204684309798
Iteración 500/5000, Costo: 3330.34326900612
Iteración 600/5000, Costo: 3113.4068208979297
Iteración 700/5000, Costo: 2955.0649256792076
Iteración 800/5000, Costo: 2824.297529918066
Iteración 900/5000, Costo: 2700.576858774634
Iteración 1000/5000, Costo: 2571.093634467427
Iteración 1100/5000, Costo: 2431.9966518821147
Iteración 1200/5000, Costo: 2287.7600800194314
Iteración 1300/5000, Costo: 2145.148698918832
Iteración 1400/5000, Costo: 2009.3272152835045
Iteración 1500/5000, Costo: 1883.2345604888446
Iteración 1600/5000, Costo: 1767.8948790956215
Iteración 1700/5000, Costo: 1662.9243026388392
Iteración 1800/5000, Costo: 1567.3887330107877
Iteración 1900/5000, Costo: 1480.4369181659174
Iteración 2000/5000, Costo: 1401.3897799360327
Iteración 2100/5000, Costo: 1329.6467067652757
Iteración 2200/5000, Costo: 1264

In [14]:
# Calcular predicciones con matrices optimizadas
predictions = mu + b_u_optimizado.reshape(-1, 1) + b_i_optimizado.reshape(1, -1) + np.dot(X_optimizado, W_optimizado.T)



#predicciones para un usuario
usuario_id = 2
predicciones_usuario = predictions[:, usuario_id]


#Películas recomendadas para un usurio
peliculas_no_vistas = np.where(matrizBinaria[:, usuario_id] == 0)[0]
predicciones_no_vistas = predictions[peliculas_no_vistas, usuario_id]




In [15]:
def recomendar_top_n(predictions, user_id, matrizBinaria, mapeo_IndiceAPelicula, n):

    peliculas_no_vistas = np.where(matrizBinaria[:, user_id] == 0)[0]

    predicciones_no_vistas = predictions[peliculas_no_vistas, user_id]


    indices_top = peliculas_no_vistas[np.argsort(predicciones_no_vistas)[::-1][:n]]

    recomendaciones = [mapeo_IndiceAPelicula[idx] for idx in indices_top]

    ##acceder al titulo
    recomendaciones = nombresDePelis[nombresDePelis['movieId'].isin(recomendaciones)]['title'].tolist()


    return recomendaciones

In [16]:
for i in range(1,11):
  print(recomendar_top_n(predictions, i ,matrizBinaria, mapeo_IndiceAPelicula, 5))


['Dangerous Minds (1995)', 'Baby-Sitters Club, The (1995)', 'Legend (1985)', 'Mad Max (1979)', 'Divided We Fall (Musíme si pomáhat) (2000)']
['Before and After (1996)', 'That Thing You Do! (1996)', 'Austin Powers: International Man of Mystery (1997)', 'Welcome to Woop-Woop (1997)', 'Wonder Boys (2000)']
['Boys on the Side (1995)', 'Junior (1994)', 'Sword in the Stone, The (1963)', 'Saturn 3 (1980)', 'Grumpy Old Men (1993)']
['Austin Powers: International Man of Mystery (1997)', 'Game, The (1997)', 'Piranha (1978)', 'Chocolat (2000)', 'L.I.E. (2001)']
['Sword in the Stone, The (1963)', 'Great Mouse Detective, The (1986)', 'Mummy, The (1999)', 'Saturn 3 (1980)', 'Shutter Island (2010)']
['Sword in the Stone, The (1963)', 'Platoon (1986)', 'Monty Python and the Holy Grail (1975)', "Can't Hardly Wait (1998)", 'Town, The (2010)']
['To Die For (1995)', 'Reckless (1995)', 'Tommy Boy (1995)', 'Sword in the Stone, The (1963)', 'Sweet Hereafter, The (1997)']
['Apollo 13 (1995)', 'Junior (1994)',

### Resultados y Análisis
#### Sistema de Recomendación en Acción
El sistema genera recomendaciones personalizadas para cada usuario basándose en las predicciones del modelo. Para un usuario dado, el proceso implica:

Identificar las películas que el usuario aún no ha visto
Predecir la calificación que el usuario daría a cada una de estas películas
Ordenar las películas según la calificación predicha
Recomendar las N películas con mayor puntuación predicha

La función recomendar_top_n implementa esta lógica y devuelve los IDs de las películas recomendadas.

Al examinar las recomendaciones generadas para los primeros usuarios, se observan algunos patrones interesantes:

Ciertas películas aparecen frecuentemente en las recomendaciones, lo que sugiere que son películas populares o de alta calidad que el modelo considera ampliamente recomendables.

Existe variación entre las recomendaciones para diferentes usuarios. Esto indica que el modelo captura algunas preferencias personales.